# Instalando bibliotecas

In [1]:
!pip install requests

In [3]:
import requests
import pandas as pd
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from datetime import date

# Criando função API

In [23]:
def buscar_vacina_aplicada_df(cidade, uf):
  #Parâmetros 
  parameters = {'size':10000,  'query':{'bool':{'must':[
                                                        {'match':{'estabelecimento_uf': uf.upper()}},
                                                        {'match':{'estabelecimento_municipio_nome':cidade.upper()}}
                                                         ]}}}
  
  #
  response = requests.get("https://imunizacao-es.saude.gov.br/_search?scroll=1m&pretty", 
                        auth=('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'), json = parameters)
  
  scroll_id = response.json()['_scroll_id']

  tamanho = len(response.json()['hits']['hits'])

  print(tamanho)

  vacinas_municipio = response.json()['hits']['hits']


  while (tamanho !=0):

    parameters_2 = {'scroll':'1m', 'scroll_id': scroll_id }

    response_2 = requests.post("https://imunizacao-es.saude.gov.br/_search/scroll", 
                            auth=('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'), json = parameters_2)
    
    vacinas_municipio = vacinas_municipio + (response_2.json()['hits']['hits'])
    
    scroll_id_2 = response_2.json()['_scroll_id']

    tamanho = len(response_2.json()['hits']['hits'])

    print(tamanho)
  

  vacinas_municipio_df = pd.DataFrame(pd.DataFrame.from_dict(vacinas_municipio)['_source'].to_list())


  return vacinas_municipio_df